<a href="https://colab.research.google.com/github/sendmail123/DataScience/blob/master/Pytorch/DQN/environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Mar 13 22:57:23 2020

@author: vishalsingh
"""

import numpy as np

#BUILDING THE ENVIRONMENT OF THE CLASS 

class Environment(object):
    
    #INTRODUCING AND INITIALIZING ALL THE VARIABLES AND PARAMETRES OF THE ENVIRONMENT
    
    def __init__(self, optimal_temprature = (18.0,24.0), initial_month=0, initial_number_users=10, initial_rate_data=60):
        self.monthly_atmospheric_temprerature = [1.0,5.0,7.0,10.0,11.0,20.0,23.0,24.0,22.0,10.0,5.0,1.0]
        self.initial_month = initial_month
        self.atmospheric_temprature = self.monthly_atmospheric_temprerature[initial_month]
        self.optimal_temprature = optimal_temprature
        self.min_temprature = -20
        self.max_temprature = 80
        self.min_number_users=10
        self.max_number_users=100
        self.max_update_users=5
        self.min_rate_data=5
        self.max_rate_data=300
        self.max_update_data=10
        self.initial_number_users=initial_number_users
        self.current_number_users=initial_number_users
        self.initial_rate_data=initial_rate_data
        self.current_rate_data=initial_rate_data
        self.intrinsic_temprature = self.atmospheric_temprature + 1.25 * self.current_number_users + 1.25 * self.current_rate_data
        self.temprature_ai = self.intrinsic_temprature
        self.temparture_noai = (self.optimal_temprature[0] + optimal_temprature[1])/2.0
        self.total_energy_ai = 0.0
        self.total_energy_noai = 0.0
        self.reward = 0.0
        self.game_over = 0
        self.train = 1
        
    #MAKING A METHOD THAT UPDATES THE ENVIRONMENT RIGHT AFTER THE AI PLAYS AN ACTION
    def update_env(self,direction,energy_ai,month):
        
        # GETTING THE REWARD
        
        #Computing the energ spent by the server when there is no AI
        energy_noai = 0
        
        if (self.temparture_noai < self.optimal_temprature[0]):
            energy_noai = self.optimal_temprature[0] - self.temparture_noai
            self.temparture_noai = self.optimal_temprature[0]
        elif (self.temparture_noai > self.optimal_temprature[1]):
            energy_noai = self.temparture_noai - self.optimal_temprature[1]
            self.temparture_noai = self.optimal_temprature[1]
            
        # Computing the Reward
        
        self.reward = energy_noai - energy_ai
        
        # Scale the Reward
        self.reward = 1e-3 * self.reward
        
        # GETTING THE NEXT STATE
        
        #Updating the atmospheric temprature
        self.atmospheric_temprature = self.monthly_atmospheric_temprerature[month]
        
        #Updating the number of user
        self.current_number_users += np.random.randint(-self.max_update_users, self.max_update_users)
        
        if(self.current_number_users>self.max_number_users):
            self.current_number_users = self.max_number_users
        elif(self.current_number_users<self.min_number_users):
            self.current_number_users = self.min_number_users
            
        #Updating the rate of Data transmission
        self.current_rate_data += np.random.randint(-self.max_update_data, self.max_update_data)
        
        if(self.current_rate_data>self.max_rate_data):
            self.current_rate_data = self.max_rate_data
        elif(self.current_rate_data<self.min_rate_data):
            self.current_rate_data = self.min_rate_data
            
        # Computing the delta of intrinisc temprature
        past_intrinsic_temprature = self.intrinsic_temprature
        self.intrinsic_temprature = self.atmospheric_temprature + 1.25 * self.current_number_users + 1.25 * self.current_rate_data
        
        delta_intrinsic_temprature = self.intrinsic_temprature - past_intrinsic_temprature
        
        #Computing the delta of temprature caused by AI
        if(direction == -1):
            delta_temprature_ai = -energy_ai
        elif(direction == 1):
            delta_temprature_ai = energy_ai
            
        #Update the temprature of the server when there is AI
        
        self.temprature_ai = self.temprature_ai + delta_intrinsic_temprature + delta_temprature_ai
        
         #Update the temprature of the server when there is no AI
        self.temprature_ai = self.temprature_ai + delta_intrinsic_temprature + delta_temprature_ai
         
         #GETTING GAME OVER
        if (self.temprature_ai < self.min_temprature):
             
             if(self.train == 1):
                 self.game = 1
             else:
                self.temprature_ai = self.optimal_temprature[0] 
                self.total_energy_ai = self.total_energy_ai +  self.optimal_temprature[0] - self.temprature_ai
                
        elif(self.temprature_ai > self.max_temprature):
            if(self.train == 1):
                self.game = 1
            else:
              self.temprature_ai = self.optimal_temprature[1]
              self.total_energy_ai = self.total_energy_ai + self.temprature_ai - self.optimal_temprature[1] 
              
        #UPDATING THE SCORES
        
        #Updating the total energy spent by AI
        
        self.total_energy_ai += energy_ai
        #Updating the total energy spent by server's cooling system when there is no AI
        self.total_energy_noai += energy_noai 
        
        #SCALE THE NEXT STATE
        scaled_temprature_ai = (self.temprature_ai - self.min_temprature)/(self.max_temprature - self.min_temprature)
        scaled_number_users = (self.current_number_users - self.min_number_users)/(self.max_number_users - self.min_number_users)
        scaled_rate_date = (self.current_rate_data - self.min_rate_data)/(self.max_rate_data - self.min_rate_data)
        next_state = np.matrix([scaled_temprature_ai,scaled_number_users,scaled_rate_date])
        
        #RETURN THE NEXT STATE REWARD AND GAME OVER
        
        return next_state, self.reward, self.game_over
    
    #MAKING A METHOD THAT RESET THE ENVIRONMENT
    
    def reset(self,new_month,):
        self.atmospheric_temprature = self.monthly_atmospheric_temprerature[new_month]
        self.initial_month = new_month
        self.current_number_users = self.initial_number_users
        self.current_rate_data=self.initial_rate_data
        self.intrinsic_temprature = self.atmospheric_temprature + 1.25 * self.current_number_users + 1.25 * self.current_rate_data
        self.temprature_ai = self.intrinsic_temprature
        self.temparture_noai = (self.optimal_temprature[0] + self.optimal_temprature[1])/2.0
        self.total_energy_ai = 0.0
        self.total_energy_noai = 0.0
        self.reward = 0.0
        self.game_over = 0
        self.train = 1
        
        
    #MAKING A METHOD THAT GIVES US AT ANY TIME CURRENT STATE, REWARD AND WHETHER THE GAME IS OVER
    
    def observe(self):
        scaled_temprature_ai = (self.temprature_ai - self.min_temprature)/(self.max_temprature - self.min_temprature)
        scaled_number_users = (self.current_number_users - self.min_number_users)/(self.max_number_users - self.min_number_users)
        scaled_rate_date = (self.current_rate_data - self.min_rate_data)/(self.max_rate_data - self.min_rate_data)
        current_state = np.matrix([scaled_temprature_ai,scaled_number_users,scaled_rate_date])
        return current_state, self.reward, self.game_over
